Esta notebook contiene bloques de código útiles para realizar Q-learning en el entorno "Taxi"

In [1]:
import numpy as np
import random
from taxi_env_extended import TaxiEnvExtended
import wandb
from tqdm import tqdm


In [2]:
env = TaxiEnvExtended()

Obtener la cantidad de estados y acciones

In [3]:
actions = env.action_space.n
states = env.observation_space.n

Inicialización de la tabla Q

In [4]:
Q = np.zeros((states, actions))
Q

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

Obtención de la acción a partir de la tabla Q

In [6]:
def optimal_policy(state, Q):
    action = np.argmax(Q[state])
    return action

Epsilon-Greedy Policy

In [7]:
def epsilon_greedy_policy(state, Q, epsilon=0.1, randomSeed=0):
    random.seed(randomSeed)
    explore = np.random.binomial(1, epsilon)
    if explore:
        action = env.action_space.sample()
        
    # exploit
    else:
        action = np.argmax(Q[state])
        
    return action

Ejemplo de episodio 

In [8]:
obs,_ = env.reset()
print(obs)
done = False
total_reward = 0
step_count = 0
while not done:
    state = obs
    action = epsilon_greedy_policy(state, Q, 0.5)
    obs, reward, done, _, _ = env.step(action)
    total_reward += reward
    step_count += 1
    print('->', state, action, reward, obs, done)
    env.render()
print('total_reward', total_reward)
print('total_steps', step_count)

186
-> 186 4 -10 186 False
-> 186 0 -1 286 False
-> 286 2 -1 286 False
-> 286 0 -1 386 False
-> 386 0 -1 486 False
-> 486 0 -1 486 False
-> 486 0 -1 486 False
-> 486 0 -1 486 False
-> 486 0 -1 486 False
-> 486 0 -1 486 False
-> 486 4 -10 486 False
-> 486 0 -1 486 False
-> 486 1 -1 386 False
-> 386 0 -1 486 False
-> 486 0 -1 486 False
-> 486 5 -10 486 False
-> 486 0 -1 486 False
-> 486 5 -10 486 False
-> 486 5 -10 486 False
-> 486 0 -1 486 False
-> 486 0 -1 486 False
-> 486 0 -1 486 False
-> 486 0 -1 486 False
-> 486 0 -1 486 False
-> 486 0 -1 486 False
-> 486 0 -1 486 False
-> 486 0 -1 486 False
-> 486 1 -1 386 False
-> 386 0 -1 486 False
-> 486 0 -1 486 False
-> 486 0 -1 486 False
-> 486 2 -1 486 False
-> 486 4 -10 486 False
-> 486 1 -1 386 False
-> 386 3 -1 366 False
-> 366 2 -1 386 False
-> 386 0 -1 486 False
-> 486 0 -1 486 False
-> 486 2 -1 486 False
-> 486 1 -1 386 False
-> 386 5 -10 386 False
-> 386 0 -1 486 False
-> 486 5 -10 486 False
-> 486 2 -1 486 False
-> 486 0 -1 486 Fals

In [ ]:
from Agent import Agent
num_episodes = 1000
wandb.init(project="taxi",
           config={
                "alpha": 0.1,
                "epsilon": 0.5,
                "Num_episodes_train": num_episodes,
                "randomSeed": 0,
           })
agent = Agent(env, gamma=0.9, Qtable=Q, alpha=0.1, epsilon=0.5,randomSeed=0, policy_func=epsilon_greedy_policy) 

##agent.train(num_episodes)



In [7]:
from Agent import Agent
agent = Agent(env, gamma=0.7, Qtable=Q, alpha=0.99, epsilon=0.1,randomSeed=45)

wandb.init(project="taxi",
           config={
               "gamma": 0.7, 
                "alpha": 0.99,
                "epsilon": 0.1,
                "Num_episodes_train":1000,
                "randomSeed": 45,
           })
for t in range(10):
    ##epsilonDecay epsilon = epsilon * (10 - t) / 10
    trainValue = agent.train(100)
    playValue = agent.play(10)
    wandb.log({"trainValue": trainValue, "playValue": playValue, "t" : t})

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: joaquinrodriguezcaussi (intart). Use `wandb login --relogin` to force relogin


100%|██████████| 100/100 [01:17<00:00,  1.29it/s]


In [9]:
from Agent import Agent
import numpy as np
import random
from taxi_env_extended import TaxiEnvExtended
import wandb
from tqdm import tqdm

def train():
    env = TaxiEnvExtended()
    Q = np.zeros((env.observation_space.n, env.action_space.n))  # Inicializa tu tabla Q aquí

    wandb.init()
    config = wandb.config

    agent = Agent(env, Qtable=Q, gamma=config.gamma, alpha=config.alpha, epsilon=config.epsilon, randomSeed=config.randomSeed)
    # Entrena el agente y registra las métricas
    for t in range(10):
        trainValue = agent.train(100)
        playValue = agent.play(10)
        wandb.log({"trainValue": trainValue, "playValue": playValue, "t" : t})

sweep_config = {
        'name': 'my-sweep',
        'method': 'bayes',
        'metric': {
            'name': 'trainValue',
            'goal': 'maximize'
        },
        'parameters': {
            'alpha': {
                'distribution': 'uniform',
                'min': 0.5,
                'max': 0.99
            },
            'epsilon': {
                'distribution': 'uniform',
                'min': 0.6,
                'max': 0.99
            },
            'gamma': {
                'distribution': 'uniform',
                'min': 0.5,
                'max': 0.99
            },
            'randomSeed': {
                'distribution': 'int_uniform',
                'min': 1,
                'max': 100
            }
        }
    }
entity = "intart"
project = "taxi"
#sweep_id = wandb.sweep(sweep_config, entity=entity, project=project)
sweep_id = "gah9p0fe"

wandb.agent(sweep_id, function=train, count=100, entity=entity, project=project)



Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
400 response executing GraphQL.
{"errors":[{"message":"Sweep intart/taxi/gah9p0fe is not running","path":["createAgent"]}],"data":{"createAgent":null}}
wandb: ERROR Error while calling W&B API: Sweep intart/taxi/gah9p0fe is not running (<Response [400]>)


UsageError: Sweep intart/taxi/gah9p0fe is not running

: 